# regression
- 设置不同threshold找motif


In [1]:
def get_substring_indices(input_string):
    # 输入一个字符串，输出一个列表，列表中的元素为该字符串的子串的index区间
    indices = []
    length = len(input_string)
    
    for i in range(length):
        for j in range(i+4,  min(i+10, length+1)):
            indices.append([i, j])
    
    return indices


def filterSubstring(grad:list, alpha:float, indexLs:list, seq:str):
    alpha_grad = [v if v>=alpha else 0 for v in grad]
    finalLs = []
    for i in indexLs:
        subGrad = alpha_grad[i[0]:i[1]]
        flag = subGrad.count(0)
        if flag <= int((i[1]-i[0])/2):
            subSeq = list(seq[i[0]:i[1]])
            for j in range(len(subSeq)):
                if subGrad[j] == 0:
                    subSeq[j] = 'X'
            if subSeq[0]!= 'X' and subSeq[-1]!= 'X':
                finalLs.append(''.join(subSeq))


    return finalLs


def filterSubstring2(grad:list, alpha:float, indexLs:list, seq:str):
    alpha_grad = [v if v>=alpha else 0 for v in grad]
    finalLs = []
    for i in indexLs:
        subGrad = alpha_grad[i[0]:i[1]]
        flag = subGrad.count(0)
        if flag <= int((i[1]-i[0])/2):
            subSeq = list(seq[i[0]:i[1]])
            for j in range(len(subSeq)):
                if subGrad[j] == 0:
                    subSeq[j] = 'X'
            if subSeq[0]!= 'X' and subSeq[-1]!= 'X':
                finalLs.append(''.join(subSeq))


    return finalLs


def find_motif(input_string,grad,alpha):
    result = get_substring_indices(input_string)
    ls = filterSubstring(grad,alpha=alpha, indexLs = result, seq=input_string)

    return ls


# 使用示例
input_string = "abcdesdkjlgf"
grad = [.4,.6,.4,0,.4,.6,.4,0,0,.4,.6,.4,]
result = get_substring_indices(input_string)
ls = filterSubstring(grad,alpha = 0.4, indexLs = result, seq=input_string)
print(ls)
ls1 = find_motif(input_string=input_string,grad = grad,alpha=0.4)
print(ls1)


['abcXe', 'abcXes', 'abcXesd', 'bcXe', 'bcXes', 'bcXesd', 'bcXesdXXl', 'cXes', 'cXesd', 'cXesdXXl', 'cXesdXXlg', 'esdXXl', 'esdXXlg', 'esdXXlgf', 'sdXXl', 'sdXXlg', 'sdXXlgf', 'dXXl', 'dXXlg', 'dXXlgf']
['abcXe', 'abcXes', 'abcXesd', 'bcXe', 'bcXes', 'bcXesd', 'bcXesdXXl', 'cXes', 'cXesd', 'cXesdXXl', 'cXesdXXlg', 'esdXXl', 'esdXXlg', 'esdXXlgf', 'sdXXl', 'sdXXlg', 'sdXXlgf', 'dXXl', 'dXXlg', 'dXXlgf']


In [3]:
# 添加NBT attention模型
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '2,3'
import torch

import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker
import math

# 下面这几行是为了调用NBT att模型
from keras.models import load_model
from numpy import loadtxt, savetxt
import re

from Attention import Attention_layer


import warnings
warnings.filterwarnings("ignore")

import requests
import re
import json
from bs4 import BeautifulSoup

url = "https://pepcalc.com/ppc.php"

mydata = json.loads(
    '{"hideInputFields": "no","nTerm": "(NH2-)","cTerm": "(-COOH)","aaCode": 0,"disulphideBonds": "","sequence": ""}')

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Content-Length': '<calculated when request is sent>'
}


mydict = {'A':0,'C':1,'D':2,'E':3,'F':4,'G':5,'H':6,'I':7,'K':8,'L':9,'M':10,'N':11,'P':12,'Q':13,'R':14,'S':15,'T':16,'V':17,'W':18,'Y':19}
myInvDict = dict([val, key] for key, val in mydict.items())
sigmoid = torch.sigmoid


NBTdict = {'A':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,'L':10,'M':11,'N':12,'P':13,'Q':14,'R':15,'S':16,'T':17,'V':18,'W':19,'Y':20}




MAX_MIC = math.log10(8192)
max_mic_buffer = 0.1
My_MAX_MIC = math.log10(600)


def CosineSimilarity(tensor_1, tensor_2):
    tensor_1 = tensor_1.squeeze()
    tensor_2 = tensor_2.squeeze()

    normalized_tensor_1 = tensor_1 / tensor_1.norm(dim=-1, keepdim=True)
    normalized_tensor_2 = tensor_2 / tensor_2.norm(dim=-1, keepdim=True)
    return (normalized_tensor_1 * normalized_tensor_2).sum()
    
def seq2num(seq):
    
    seqlist = list(seq)

    length = len(seq)
    result = re.findall(r'[BJOUXZ]',seq)

    if result:
        return 

    # 否则正常返回
    else:
        numlist = [NBTdict[char.upper()] for char in seqlist]
        
        zeroPad = [0 for i in range(300-length)]
        zeroPad.extend(numlist)
        zeroPad = np.array(zeroPad)
        
        return zeroPad


def dataProcessPipeline(seq):
    # 本函数先把序列转化为0-19组成的序列，然后onehot变化，再padding
    # 同时返回padding后的序列以及mask
    #print('ori seq',seq)
    testest = seq
    num_seq = [mydict[character.upper()] for character in seq]

    seq = np.array(num_seq,dtype=int)
    len = seq.shape[0]
    torch_seq = torch.tensor(seq)

    if torch.sum(torch_seq[torch_seq<0])!=0:
        print(torch_seq[torch_seq<0])
        print('wrong seq:',seq)
        print(testest)

    onehotSeq = torch.nn.functional.one_hot(torch_seq,num_classes=20)
    pad = torch.nn.ZeroPad2d(padding=(0,0,0,100-len))
    mask = np.zeros(100,dtype = int)
    mask[len:]=1
    mask = torch.tensor(mask)
    pad_seq = pad(onehotSeq) 
    
    
    return pad_seq,mask


def num2onehot(array2d):
    result = torch.zeros_like(array2d)
    index = torch.argmax(array2d,dim = -1)
    for i in range(index.shape[0]):
        result[i,index[i]] = 1

    return result


class TrainDataset(Dataset):
    def __init__(self,data_path,transform = dataProcessPipeline):

        df = pd.read_csv(data_path,header=0)
        self.df = df
        self.seqs = list(self.df['sequence'])

        #print(self.seqs.shape)
        self.values = self.df['value']
        # 数据集的单边阈值设置
        self.values[self.values>MAX_MIC] = MAX_MIC
        self.values = list(self.values)
        #print(self.labels.shape)
        self.transform = transform


    def __getitem__(self,idex):
        seq = self.seqs[idex]
        num_seq, mask = self.transform(seq)
        label = self.values[idex]


        return num_seq, mask, label, seq

    def __len__(self):
        return len(self.seqs)


class TestDataset(Dataset):
    def __init__(self,data_path,transform = dataProcessPipeline):
        self.df = pd.read_csv(data_path,header=0)
        self.seqs = self.df['Sequence']

        self.transform = transform


    def __getitem__(self,idex):
        seq = self.seqs[idex]
        num_seq, mask = self.transform(seq)

        return num_seq, mask, seq

    def __len__(self):
        return len(self.seqs)



class PositionalEncoding(nn.Module):
     def __init__(self, len, d_model=20, dropout=0):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(len, d_model)
        position = torch.arange(0, len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                                * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        #pe = pe.unsqueeze(0).transpose(0, 1)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

     def forward(self, x):
        x = x + self.pe
        #x = x + self.pe[:,:x.size(0), :]
        return x



pe = PositionalEncoding(len=100,d_model = 20)


class AttentionNetwork(nn.Module):
    
    def __init__(self,batch_size=128,embedding_size=20,num_tokens=100,num_classes=1,num_heads=4):
        super(AttentionNetwork,self).__init__()
        self.pe = PositionalEncoding(len=num_tokens,d_model = embedding_size)
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.num_tokens = num_tokens
        self.num_classes = num_classes
        self.num_heads = num_heads
        # self.hidden1 = 20
        self.hidden1 = 20
        self.hidden2 = 60
        self.hidden3 = 20
        self.dropout = 0.2

        # self.hidden2 = 100
        # self.hidden3 = 50
        # self.hidden4 = 20
        # self.dropout = 0.2
        self.relu = nn.ReLU()

        self.LN = nn.LayerNorm(normalized_shape = self.hidden1)
        self.fc1 = nn.Linear(self.embedding_size,self.hidden1)

        # self.qfc = nn.Linear(self.hidden1,self.hidden1)
        # self.kfc = nn.Linear(self.hidden1,self.hidden1)
        # self.vfc = nn.Linear(self.hidden1,self.hidden1)

        self.multihead_att = nn.MultiheadAttention(embed_dim=self.hidden1,num_heads = self.num_heads,batch_first=1,dropout=self.dropout)
        # 我这里先不用maxpool了
        self.flatten = nn.Flatten()
        self.fc2 = nn.Linear(self.hidden1*self.num_tokens,self.hidden2)
        self.fc3 = nn.Linear(self.hidden2,self.hidden3)
        self.new_fc4 = nn.Linear(self.hidden3,self.num_classes)
        # self.fc4 = nn.Linear(self.hidden3,self.hidden4)
        # self.fc5 = nn.Linear(self.hidden4,self.num_classes)
        self.dropout = nn.Dropout(self.dropout)
        self.softmax = nn.functional.softmax


    #这里的X当作对象 有 embedding 和 mask
    def forward(self,x,mask):
        x = self.pe(x)
        x = self.fc1(x)


        mask = mask.to(torch.bool)
        x, w1= self.multihead_att.forward(x,x,x,key_padding_mask=mask)
        # x, w1= self.multihead_att.forward(x,x,x,key_padding_mask=mask)
        # x, w1= self.multihead_att.forward(x,x,x,key_padding_mask=mask)


        # x = self.LN(x)
        # [N 100 20]

        
        
        #print(type(x),x.size)
        # x = torch.tensor(x)
        x = self.flatten(x)
        x = self.fc2(x)
        x = self.dropout(x)
        
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.new_fc4(x)


        #return x, w1, w2, w3, w4
        return x

Using TensorFlow backend.
/home/user2/anaconda3/envs/AMP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user2/anaconda3/envs/AMP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user2/anaconda3/envs/AMP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user2/anaconda3/envs/AMP/l

In [3]:
model_list = {
    # 
    'myAttention': '../../NewModel3_21_output1_Regression/myAttention/mean_0_changeTH_0_0/_AMP0.629_total0.543.pth',

}

test_model_list = model_list

opt_seqls = [

'LWWRKAKWKRKIAKRMIRVIGAAKI',                                                                                                                            
'KWLGAFGKMRKIAIRLRLKRKKAF',                                                                                                                             
'WWRLWKTLLKAPKKLTGLRRW',                                                                                                                             
'RKLKKLRWRAGMMYKYVKLK',
'MRFPWKHWWKKWKWWWKKKR',
'MKKARFWWWVAWKKLLRKKA'
]

labels = [
'myAttention9',
'myAttention7',
'CNN8',
'myAttention9',
'myAttention13',
'myAttention9'
]

label_dict = {opt_seqls[i]:labels[i] for i in range(len(opt_seqls))}

import json
import os
import numpy as np

# 指定之前保存的 JSON 文件名
file_name = "/home/user2/pj/AMP_2/attention_model/test_motif/score/seq_score_9_7_reg.json"


with open(file_name, "r") as file:
    loaded_data = json.load(file)

score_seq_dict = {}
for item in loaded_data:
    seq = item['seq']
    # print(seq)
    score = np.array(item['score'])
    score_seq_dict[seq] = score
    # print(score.shape)
    # draw_weight2(score,seq,'layer1')


df_ls = []
from matplotlib.colors import Normalize
norm = Normalize(vmin=-0.2, vmax=0.2)
for seq in opt_seqls:
    tseq = seq

    # ModelNameList = ['CNN','Transformer','myAttention','RCNN']
    ModelNameList = model_list.keys()

    gradient = {}
    

    oriseq = tseq

    # 在当前目录生成 该序列的csv 文件
    df = pd.DataFrame(columns = ['Sequence','Length','label'])
    items = [{'Sequence':oriseq,'Length':len(oriseq)}]
    df = df.append(items,ignore_index = 1)
    df.to_csv(oriseq+'.csv',index = False)


    SeqPath = oriseq+'.csv'



    testData1 = TrainDataset(data_path = r'../../myRegressionData/all_balance/mean/test.csv')
    test_loader1 = DataLoader(dataset=testData1, batch_size=4,drop_last=True)


    for modelName in ModelNameList:

        # modelName = 'myAttention'  # to change
        iternum = 0

        testData = TestDataset(data_path = SeqPath)
        test_loader = DataLoader(dataset=testData, batch_size=1)
        attmodel = torch.load(model_list[modelName])


        attmodel.cuda()
        attmodel.zero_grad()

        print(modelName,"_V2:")

            
        for data in test_loader: #序列优化 stratergy 1: 全局都用ensamble作为优化目标
            resultList = []
            # ensamble_values = []
            resultSeq = [oriseq]
            outMIC = []
            # attmodel.zero_grad()
            inputs,masks, seqs = data

            inputs = inputs.float()
            masks = masks.float()
            
            # inputs = inputs.cuda()
            # inputs.requires_grad = True
            masks = masks.cuda()
            print(seqs[0])

            if modelName == 'RCNN':
                attmodel.train()
            else:
                attmodel.eval()
    
            attmodel.zero_grad()
            stdev_spread = 0.1
            n_samples = 25
            x = inputs[0].detach()
            stdev = stdev_spread * (torch.max(x)-torch.min(x))
            x = x.numpy() 
            total_grad = np.zeros_like(x)
            for i in range(n_samples):

                    # final.append(xx)
                noise = np.random.normal(0,stdev,x.shape).astype(np.float32)
                x_plus_noise = x + noise
                
                x_plus_noise = torch.from_numpy(x_plus_noise).cuda()
                # x_plus_noise = x_plus_noise + 0.1
                x_plus_noise[masks[0]==1] = 0
                x_plus_noise = Variable(torch.unsqueeze(x_plus_noise,dim=0), requires_grad = True)
                # final.append(x_plus_noise)
                if i==0:
                    x_plus_noise = Variable(inputs.cuda(), requires_grad = True)
            
                if modelName == 'lstm_att' or modelName == 'RCNN'or modelName == 'CNN' or modelName == 'Transformer':
                    out = attmodel(x_plus_noise)
                else:
                    out = attmodel(x_plus_noise,masks)

                out = out.cpu()
                if 'RCNN' in modelName:
                    out = out.unsqueeze(0)

                conloss = -out

                conloss.backward()
                grad = x_plus_noise.grad
                # if i==0:
                #     print(grad)
                colindex = masks[0]==1
                grad[0][masks[0]==1] = 0
                grad = grad[0].cpu().numpy()
                
                total_grad += grad


            avg_grad = total_grad/n_samples
            myIndex = [mydict[v] for v in seqs[0]]

            mylen = 100-colindex.sum()
            # print(mylen)
            grad = avg_grad[:mylen]
            # grad = [grad[k,myIndex[k]] for k in range(len(seqs[0]))]
            mag = np.sum(abs(grad),axis=-1).tolist()

            # print(np.sum(grad))

            value = [grad[k,myIndex[k]] for k in range(len(seqs[0]))]
            # print(value)
            # max_grad = np.max(grad,axis=-1).tolist()
            # # grad  =  [value[i]/max_grad[i] for i in range(len(grad))]
            grad = [mag[k]*value[k] for k in range(len(value))]
            # grad = list(np.exp(grad))

            absGrad = np.sum(abs(np.array(grad)))
            grad = [v/absGrad for v in grad]
            # grad = [ v/sum(grad) for v in grad]
            gradient[modelName] = grad
    seq_dict = {'sequence': seq,'model':label_dict[seq]}
    for k in range(11):
        alpha = k/100
        ls = find_motif(tseq,grad,alpha)
        print(tseq,':',ls)
        print(sum(abs(np.array(grad))))
        seq_dict[alpha] = ls

    df_ls.append(seq_dict)


    # print(gradient)
    # grad = gradient['myAttention']
    # alpha = 0.05
    # ls = find_motif(tseq,grad,alpha)
    # print(tseq,':',ls)
    # print(sum(abs(np.array(grad))))


(1312, 3)

myAttention _V2:
LWWRKAKWKRKIAKRMIRVIGAAKI
LWWRKAKWKRKIAKRMIRVIGAAKI : ['LWWXK', 'LWWXKXXW', 'WWXK', 'WWXKXXW', 'WWXKXXWK', 'WXKXXW', 'WXKXXWK', 'KXXW', 'KXXWK', 'KXXWKXXI', 'WKXXI', 'WKXXIXK', 'KXXI', 'KXXIXK', 'KXXIXKXM', 'IXKXM', 'IXKXMI', 'IXKXMIR', 'IXKXMIRV', 'KXMI', 'KXMIR', 'KXMIRV', 'KXMIRVI', 'MIRV', 'MIRVI', 'IRVI', 'RVIXXXXI']
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : ['LWWXK', 'LWWXKXXW', 'WWXK', 'WWXKXXW', 'WXKXXW', 'KXXW', 'KXMI', 'KXMIR']
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : ['LWWXK', 'LWWXKXXW', 'WWXK', 'WWXKXXW', 'WXKXXW', 'KXXW']
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : ['LWWXXXXW']
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : []
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : []
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : []
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : []
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : []
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : []
0.9999999999999997
LWWRKAKWKRKIAKRMIRVIGAAKI : []
0.9999999

In [4]:
df = pd.DataFrame(df_ls)
print(df)
df.to_csv('./6peptid_threshold.csv',index=0)

                    sequence          model  \
0  LWWRKAKWKRKIAKRMIRVIGAAKI   myAttention9   
1   KWLGAFGKMRKIAIRLRLKRKKAF   myAttention7   
2      WWRLWKTLLKAPKKLTGLRRW           CNN8   
3       RKLKKLRWRAGMMYKYVKLK   myAttention9   
4       MRFPWKHWWKKWKWWWKKKR  myAttention13   
5       MKKARFWWWVAWKKLLRKKA   myAttention9   

                                                 0.0  \
0  [LWWXK, LWWXKXXW, WWXK, WWXKXXW, WWXKXXWK, WXK...   
1  [KWLG, KWLGXF, KWLGXFG, KWLGXFGK, WLGXF, WLGXF...   
2  [WWRL, WWRLW, WWRLWK, WWRLWKXL, WRLW, WRLWK, W...   
3  [KXKK, KXKKL, KXKKLR, KXKKLRW, KXKKLRWR, KKLR,...   
4  [MRFXW, MRFXWXH, MRFXWXHW, RFXW, RFXWXH, RFXWX...   
5  [MKKA, MKKAR, MKKARF, MKKARFW, MKKARFWW, KKAR,...   

                                                0.01  \
0  [LWWXK, LWWXKXXW, WWXK, WWXKXXW, WXKXXW, KXXW,...   
1  [KWLG, KWLGXF, KWLGXFXK, WLGXF, WLGXFXK, LGXF,...   
2  [WWRL, WWRLW, WWRLWK, WWRLWKXL, WRLW, WRLWK, W...   
3  [KXKK, KXKKXR, KXKKXRW, KXKKXRWR, KKXR, KKXRW,... 

In [1]:
# 仅仅设置阈值可能不够，rank呢？


import numpy as np

# 给定一个由浮点数组成的列表
data = [3.2, 1.8, 2.5, 4.0, 0.5]

# 使用numpy的argsort函数获取排序后的索引
sorted_indices = np.argsort(data)

print("排序后的索引：", sorted_indices)


排序后的索引： [4 1 2 0 3]


In [2]:
import numpy as np

# 给定一个由浮点数组成的列表
data = [3.2, 1.8, 2.5, 4.0, 0.5]

# 使用numpy的argsort函数获取排序后的索引
sorted_indices = np.argsort(data)

# 创建一个新的列表，其中的元素表示原列表对应位置的大小排序
size_sorted_list = [0] * len(data)

for i, idx in enumerate(sorted_indices):
    size_sorted_list[idx] = i

print("对应位置的大小排序：", size_sorted_list)


对应位置的大小排序： [3, 1, 2, 4, 0]


In [4]:
data = [3.2, 1.8, 2.5, 4.0, 0.5]

# 使用sorted()函数对原列表进行排序，同时获取排序后的索引
sorted_indices = sorted(range(len(data)), key=lambda i: data[i])
# print(sorted_indices)
# 创建一个新的列表，其中的元素表示原列表对应位置的大小排序
size_sorted_list = [0] * len(data)

for i, idx in enumerate(sorted_indices):
    size_sorted_list[idx] = i

print("对应位置的大小排序：", size_sorted_list)


[4, 1, 2, 0, 3]
对应位置的大小排序： [3, 1, 2, 4, 0]


## rank的方法 + abs*value

In [4]:
import numpy as np

def get_substring_indices(input_string):
    # 输入一个字符串，输出一个列表，列表中的元素为该字符串的子串的index区间
    indices = []
    length = len(input_string)
    
    for i in range(length):
        for j in range(i+4,  min(i+9, length+1)):
            indices.append([i, j])
    
    return indices

def filterSubstring(grad:list, alpha:float, indexLs:list, seq:str):

    sorted_indices = sorted(range(len(grad)), key=lambda i: grad[i])
    size_sorted_list = [0] * len(grad)

    for i, idx in enumerate(sorted_indices):
        size_sorted_list[idx] = i
    size_sorted_list =  [v/np.max(size_sorted_list) for v in size_sorted_list]
    
    alpha_grad = [grad[i] if v>=alpha else 0 for i,v in  enumerate(size_sorted_list)]
    # print(alpha_grad)
    # alpha_grad = [v if v>=alpha else 0 for v in grad]
    finalLs = []
    for i in indexLs:
        subGrad = alpha_grad[i[0]:i[1]]
        flag = subGrad.count(0)
        if flag <= int(len(subGrad)/2)-1:
            subSeq = list(seq[i[0]:i[1]])
            for j in range(len(subSeq)):
                if subGrad[j] == 0:
                    subSeq[j] = 'X'
            if subSeq[0]!= 'X' and subSeq[-1]!= 'X':
                finalLs.append(''.join(subSeq))


    return finalLs

def find_motif(input_string,grad,alpha):
    result = get_substring_indices(input_string)
    ls = filterSubstring(grad,alpha=alpha, indexLs = result, seq=input_string)

    return ls


# 使用示例
input_string = "abcdesdkjlgf"
grad = [.4,.6,.4,0,.4,.6,.4,0,0,.4,.6,.4,]
result = get_substring_indices(input_string)
ls = filterSubstring(grad,alpha = 0.4, indexLs = result, seq=input_string)
print(ls)
ls1 = find_motif(input_string=input_string,grad = grad,alpha=0.4)
print(ls1)

['bXXesd', 'esdXXl', 'esdXXlg', 'esdXXlgf', 'sdXXlg', 'sdXXlgf', 'dXXlgf']
['bXXesd', 'esdXXl', 'esdXXlg', 'esdXXlgf', 'sdXXlg', 'sdXXlgf', 'dXXlgf']


In [5]:
# 添加NBT attention模型
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '2,3'
import torch

import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker
import math

# 下面这几行是为了调用NBT att模型
from keras.models import load_model
from numpy import loadtxt, savetxt
import re

from Attention import Attention_layer


import warnings
warnings.filterwarnings("ignore")

import requests
import re
import json
from bs4 import BeautifulSoup

url = "https://pepcalc.com/ppc.php"

mydata = json.loads(
    '{"hideInputFields": "no","nTerm": "(NH2-)","cTerm": "(-COOH)","aaCode": 0,"disulphideBonds": "","sequence": ""}')

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Content-Length': '<calculated when request is sent>'
}


mydict = {'A':0,'C':1,'D':2,'E':3,'F':4,'G':5,'H':6,'I':7,'K':8,'L':9,'M':10,'N':11,'P':12,'Q':13,'R':14,'S':15,'T':16,'V':17,'W':18,'Y':19}
myInvDict = dict([val, key] for key, val in mydict.items())
sigmoid = torch.sigmoid


NBTdict = {'A':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,'L':10,'M':11,'N':12,'P':13,'Q':14,'R':15,'S':16,'T':17,'V':18,'W':19,'Y':20}




MAX_MIC = math.log10(8192)
max_mic_buffer = 0.1
My_MAX_MIC = math.log10(600)


def CosineSimilarity(tensor_1, tensor_2):
    tensor_1 = tensor_1.squeeze()
    tensor_2 = tensor_2.squeeze()

    normalized_tensor_1 = tensor_1 / tensor_1.norm(dim=-1, keepdim=True)
    normalized_tensor_2 = tensor_2 / tensor_2.norm(dim=-1, keepdim=True)
    return (normalized_tensor_1 * normalized_tensor_2).sum()
    
def seq2num(seq):
    
    seqlist = list(seq)
    # print(seq)
    length = len(seq)
    result = re.findall(r'[BJOUXZ]',seq)
    # print(result)
    # 如果序列中有这几个氨基酸，则返回空
    if result:
        return 

    # 否则正常返回
    else:
        numlist = [NBTdict[char.upper()] for char in seqlist]
        
        zeroPad = [0 for i in range(300-length)]
        zeroPad.extend(numlist)
        zeroPad = np.array(zeroPad)
        
        return zeroPad


def dataProcessPipeline(seq):
    # 本函数先把序列转化为0-19组成的序列，然后onehot变化，再padding
    # 同时返回padding后的序列以及mask
    #print('ori seq',seq)
    testest = seq
    num_seq = [mydict[character.upper()] for character in seq]

    seq = np.array(num_seq,dtype=int)
    len = seq.shape[0]
    torch_seq = torch.tensor(seq)
    if torch.sum(torch_seq[torch_seq<0])!=0:
        print(torch_seq[torch_seq<0])
        print('wrong seq:',seq)
        print(testest)
    onehotSeq = torch.nn.functional.one_hot(torch_seq,num_classes=20)
    #onehotSeq = torch.nn.functional.one_hot(c
    pad = torch.nn.ZeroPad2d(padding=(0,0,0,100-len))
    mask = np.zeros(100,dtype = int)
    mask[len:]=1
    mask = torch.tensor(mask)

    pad_seq = pad(onehotSeq) 
    
    
    return pad_seq,mask


def num2onehot(array2d):
    result = torch.zeros_like(array2d)
    index = torch.argmax(array2d,dim = -1)
    for i in range(index.shape[0]):
        result[i,index[i]] = 1

    return result


class TrainDataset(Dataset):
    def __init__(self,data_path,transform = dataProcessPipeline):
        df = pd.read_csv(data_path,header=0)
        
        
        print(str(df.shape)+'\n')
        # df = df[df['Length']<=100]
        self.df = df
        # id = self.df['Length']<100
        # self.df = self.df[id]
        # print(self.df.shape)
        #self.df = self.df.ix[1:]
        # self.seqs = list(self.df['Sequence'])
        self.seqs = list(self.df['sequence'])

        #print(self.seqs.shape)
        self.values = self.df['value']
        # 数据集的单边阈值设置
        self.values[self.values>MAX_MIC] = MAX_MIC
        self.values = list(self.values)
        #print(self.labels.shape)
        self.transform = transform


    def __getitem__(self,idex):
        seq = self.seqs[idex]
        num_seq, mask = self.transform(seq)
        label = self.values[idex]


        return num_seq, mask, label, seq

    def __len__(self):
        return len(self.seqs)


class TestDataset(Dataset):
    def __init__(self,data_path,transform = dataProcessPipeline):
        self.df = pd.read_csv(data_path,header=0)
        self.seqs = self.df['Sequence']

        self.transform = transform


    def __getitem__(self,idex):
        seq = self.seqs[idex]
        num_seq, mask = self.transform(seq)

        return num_seq, mask, seq

    def __len__(self):
        return len(self.seqs)



class PositionalEncoding(nn.Module):
     def __init__(self, len, d_model=20, dropout=0):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(len, d_model)
        position = torch.arange(0, len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                                * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        #pe = pe.unsqueeze(0).transpose(0, 1)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

     def forward(self, x):
        x = x + self.pe
        #x = x + self.pe[:,:x.size(0), :]
        return x



pe = PositionalEncoding(len=100,d_model = 20)


class AttentionNetwork(nn.Module):
    
    def __init__(self,batch_size=128,embedding_size=20,num_tokens=100,num_classes=1,num_heads=4):
        super(AttentionNetwork,self).__init__()
        self.pe = PositionalEncoding(len=num_tokens,d_model = embedding_size)
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.num_tokens = num_tokens
        self.num_classes = num_classes
        self.num_heads = num_heads
        # self.hidden1 = 20
        self.hidden1 = 20
        self.hidden2 = 60
        self.hidden3 = 20
        self.dropout = 0.2

        # self.hidden2 = 100
        # self.hidden3 = 50
        # self.hidden4 = 20
        # self.dropout = 0.2
        self.relu = nn.ReLU()

        self.LN = nn.LayerNorm(normalized_shape = self.hidden1)
        self.fc1 = nn.Linear(self.embedding_size,self.hidden1)

        # self.qfc = nn.Linear(self.hidden1,self.hidden1)
        # self.kfc = nn.Linear(self.hidden1,self.hidden1)
        # self.vfc = nn.Linear(self.hidden1,self.hidden1)

        self.multihead_att = nn.MultiheadAttention(embed_dim=self.hidden1,num_heads = self.num_heads,batch_first=1,dropout=self.dropout)
        # 我这里先不用maxpool了
        self.flatten = nn.Flatten()
        self.fc2 = nn.Linear(self.hidden1*self.num_tokens,self.hidden2)
        self.fc3 = nn.Linear(self.hidden2,self.hidden3)
        self.new_fc4 = nn.Linear(self.hidden3,self.num_classes)
        # self.fc4 = nn.Linear(self.hidden3,self.hidden4)
        # self.fc5 = nn.Linear(self.hidden4,self.num_classes)
        self.dropout = nn.Dropout(self.dropout)
        self.softmax = nn.functional.softmax


    #这里的X当作对象 有 embedding 和 mask
    def forward(self,x,mask):
        x = self.pe(x)
        x = self.fc1(x)


        mask = mask.to(torch.bool)
        x, w1= self.multihead_att.forward(x,x,x,key_padding_mask=mask)
        # x, w1= self.multihead_att.forward(x,x,x,key_padding_mask=mask)
        # x, w1= self.multihead_att.forward(x,x,x,key_padding_mask=mask)


        # x = self.LN(x)
        # [N 100 20]

        
        
        #print(type(x),x.size)
        # x = torch.tensor(x)
        x = self.flatten(x)
        x = self.fc2(x)
        x = self.dropout(x)
        
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.new_fc4(x)


        #return x, w1, w2, w3, w4
        return x

In [8]:
model_list = {
    # 
    'myAttention': '../../NewModel3_21_output1_Regression/myAttention/mean_0_changeTH_0_0/_AMP0.629_total0.543.pth',

}

test_model_list = model_list

opt_seqls = [

'LWWRKAKWKRKIAKRMIRVIGAAKI',                                                                                                                            
'KWLGAFGKMRKIAIRLRLKRKKAF',                                                                                                                             
'WWRLWKTLLKAPKKLTGLRRW',                                                                                                                             
'RKLKKLRWRAGMMYKYVKLK',
'MRFPWKHWWKKWKWWWKKKR',
'MKKARFWWWVAWKKLLRKKA'
]

labels = [
'myAttention9',
'myAttention7',
'CNN8',
'myAttention9',
'myAttention13',
'myAttention9'
]

label_dict = {opt_seqls[i]:labels[i] for i in range(len(opt_seqls))}

import json
import os
import numpy as np

# 指定之前保存的 JSON 文件名
file_name = "/home/user2/pj/AMP_2/attention_model/test_motif/score/seq_score_9_7_reg.json"


with open(file_name, "r") as file:
    loaded_data = json.load(file)

score_seq_dict = {}
for item in loaded_data:
    seq = item['seq']
    # print(seq)
    score = np.array(item['score'])
    score_seq_dict[seq] = score
    # print(score.shape)
    # draw_weight2(score,seq,'layer1')


df_ls = []
from matplotlib.colors import Normalize
norm = Normalize(vmin=-0.2, vmax=0.2)
for seq in opt_seqls:
    tseq = seq

    # ModelNameList = ['CNN','Transformer','myAttention','RCNN']
    ModelNameList = model_list.keys()

    gradient = {}
    

    oriseq = tseq

    # 在当前目录生成 该序列的csv 文件
    df = pd.DataFrame(columns = ['Sequence','Length','label'])
    items = [{'Sequence':oriseq,'Length':len(oriseq)}]
    df = df.append(items,ignore_index = 1)
    df.to_csv(oriseq+'.csv',index = False)


    SeqPath = oriseq+'.csv'



    testData1 = TrainDataset(data_path = r'../../myRegressionData/all_balance/mean/test.csv')
    test_loader1 = DataLoader(dataset=testData1, batch_size=4,drop_last=True)


    for modelName in ModelNameList:

        # modelName = 'myAttention'  # to change
        iternum = 0

        testData = TestDataset(data_path = SeqPath)
        test_loader = DataLoader(dataset=testData, batch_size=1)
        attmodel = torch.load(model_list[modelName])


        attmodel.cuda()
        attmodel.zero_grad()

        print(modelName,"_V2:")

            
        for data in test_loader: #序列优化 stratergy 1: 全局都用ensamble作为优化目标
            resultList = []
            # ensamble_values = []
            resultSeq = [oriseq]
            outMIC = []
            # attmodel.zero_grad()
            inputs,masks, seqs = data

            inputs = inputs.float()
            masks = masks.float()
            
            # inputs = inputs.cuda()
            # inputs.requires_grad = True
            masks = masks.cuda()
            print(seqs[0])

            if modelName == 'RCNN':
                attmodel.train()
            else:
                attmodel.eval()
    
            attmodel.zero_grad()
            stdev_spread = 0.1
            n_samples = 25
            x = inputs[0].detach()
            stdev = stdev_spread * (torch.max(x)-torch.min(x))
            x = x.numpy() 
            total_grad = np.zeros_like(x)
            for i in range(n_samples):

                    # final.append(xx)
                noise = np.random.normal(0,stdev,x.shape).astype(np.float32)
                x_plus_noise = x + noise
                
                x_plus_noise = torch.from_numpy(x_plus_noise).cuda()
                # x_plus_noise = x_plus_noise + 0.1
                x_plus_noise[masks[0]==1] = 0
                x_plus_noise = Variable(torch.unsqueeze(x_plus_noise,dim=0), requires_grad = True)
                # final.append(x_plus_noise)
                if i==0:
                    x_plus_noise = Variable(inputs.cuda(), requires_grad = True)
            
                if modelName == 'lstm_att' or modelName == 'RCNN'or modelName == 'CNN' or modelName == 'Transformer':
                    out = attmodel(x_plus_noise)
                else:
                    out = attmodel(x_plus_noise,masks)

                out = out.cpu()
                if 'RCNN' in modelName:
                    out = out.unsqueeze(0)

                conloss = -out

                conloss.backward()
                grad = x_plus_noise.grad
                # if i==0:
                #     print(grad)
                colindex = masks[0]==1
                grad[0][masks[0]==1] = 0
                grad = grad[0].cpu().numpy()
                
                total_grad += grad


            avg_grad = total_grad/n_samples
            myIndex = [mydict[v] for v in seqs[0]]

            mylen = 100-colindex.sum()
            # print(mylen)
            grad = avg_grad[:mylen]
            # grad = [grad[k,myIndex[k]] for k in range(len(seqs[0]))]
            mag = np.sum(abs(grad),axis=-1).tolist()

            # print(np.sum(grad))

            value = [grad[k,myIndex[k]] for k in range(len(seqs[0]))]
            # print(value)
            # max_grad = np.max(grad,axis=-1).tolist()
            # # grad  =  [value[i]/max_grad[i] for i in range(len(grad))]
            grad = [mag[k]*value[k] for k in range(len(value))]
            # grad = list(np.exp(grad))

            absGrad = np.sum(abs(np.array(grad)))
            grad = [v/absGrad for v in grad]
            # grad = [ v/sum(grad) for v in grad]
            gradient[modelName] = grad
    seq_dict = {'sequence': seq,'model':label_dict[seq]}
    for k in range(2,11):
        alpha = k/10
        ls = find_motif(tseq,grad,alpha)
        print(tseq,alpha,':',ls)
        print(sum(abs(np.array(grad))))
        seq_dict[alpha] = ls

    df_ls.append(seq_dict)


    # print(gradient)
    # grad = gradient['myAttention']
    # alpha = 0.05
    # ls = find_motif(tseq,grad,alpha)
    # print(tseq,':',ls)
    # print(sum(abs(np.array(grad))))


(1312, 3)

myAttention _V2:
LWWRKAKWKRKIAKRMIRVIGAAKI
LWWRKAKWKRKIAKRMIRVIGAAKI 0.2 : ['LWWR', 'LWWRK', 'LWWRKA', 'LWWRKAK', 'LWWRKAKW', 'WWRK', 'WWRKA', 'WWRKAK', 'WWRKAKW', 'WWRKAKWK', 'WRKA', 'WRKAK', 'WRKAKW', 'WRKAKWK', 'WRKAKWKR', 'RKAK', 'RKAKW', 'RKAKWK', 'RKAKWKR', 'KAKW', 'KAKWK', 'KAKWKR', 'KAKWKRXI', 'AKWK', 'AKWKR', 'AKWKRXI', 'KWKR', 'KWKRXI', 'WKRXI', 'WKRXIXXR', 'KRXI', 'KRXIXXRM', 'RXIXXRMI', 'IXXRMI', 'IXXRMIR', 'IXXRMIRV', 'RMIR', 'RMIRV', 'RMIRVI', 'RMIRVIG', 'RMIRVIGA', 'MIRV', 'MIRVI', 'MIRVIG', 'MIRVIGA', 'IRVI', 'IRVIG', 'IRVIGA', 'RVIG', 'RVIGA', 'RVIGAXXI', 'VIGA', 'VIGAXXI', 'IGAXXI']
1.0
LWWRKAKWKRKIAKRMIRVIGAAKI 0.3 : ['LWWXK', 'LWWXKA', 'LWWXKAXW', 'WWXK', 'WWXKA', 'WWXKAXW', 'WWXKAXWK', 'WXKA', 'WXKAXW', 'WXKAXWK', 'WXKAXWKR', 'KAXW', 'KAXWK', 'KAXWKR', 'KAXWKRXI', 'AXWK', 'AXWKR', 'AXWKRXI', 'WKRXI', 'KRXI', 'IXXXMIRV', 'MIRV', 'MIRVI', 'MIRVIG', 'MIRVIGA', 'IRVI', 'IRVIG', 'IRVIGA', 'RVIG', 'RVIGA', 'RVIGAXXI', 'VIGA', 'VIGAXXI', 'IGAXXI']
1.0
LWWRKAKWK

In [9]:
df = pd.DataFrame(df_ls)
print(df)
df.to_csv('./6peptid_threshold_rank_0.2.csv',index=0)

                    sequence          model  \
0  LWWRKAKWKRKIAKRMIRVIGAAKI   myAttention9   
1   KWLGAFGKMRKIAIRLRLKRKKAF   myAttention7   
2      WWRLWKTLLKAPKKLTGLRRW           CNN8   
3       RKLKKLRWRAGMMYKYVKLK   myAttention9   
4       MRFPWKHWWKKWKWWWKKKR  myAttention13   
5       MKKARFWWWVAWKKLLRKKA   myAttention9   

                                                 0.2  \
0  [LWWR, LWWRK, LWWRKA, LWWRKAK, LWWRKAKW, WWRK,...   
1  [KWLXXF, KWLXXFG, KWLXXFGK, WLXXFG, WLXXFGK, L...   
2  [WWRL, WWRLW, WWRLWK, WWRLWKXL, WRLW, WRLWK, W...   
3  [KXKK, KXKKL, KXKKLR, KXKKLRW, KXKKLRWR, KKLR,...   
4  [MRFXW, MRFXWK, MRFXWKH, MRFXWKHW, RFXW, RFXWK...   
5  [MKXA, MKXAR, MKXARF, MKXARFW, MKXARFWW, KXAR,...   

                                                 0.3  \
0  [LWWXK, LWWXKA, LWWXKAXW, WWXK, WWXKA, WWXKAXW...   
1  [KWLXXF, KWLXXFXK, FXKXRK, FXKXRKI, KXRK, KXRK...   
2  [WWRL, WWRLW, WWRLWK, WRLW, WRLWK, WRLWKXXL, R...   
3  [KXKK, KXKKXR, KXKKXRW, KXKKXRWR, KKXR, KKXRW,... 

## rank的方法 + abs* (value/|value|)

In [1]:
import numpy as np

def get_substring_indices(input_string):
    # 输入一个字符串，输出一个列表，列表中的元素为该字符串的子串的index区间
    indices = []
    length = len(input_string)
    
    for i in range(length):
        for j in range(i+4,  min(i+9, length+1)):
            indices.append([i, j])
    
    return indices

def filterSubstring(grad:list, alpha:float, indexLs:list, seq:str):

    sorted_indices = sorted(range(len(grad)), key=lambda i: grad[i])
    size_sorted_list = [0] * len(grad)

    for i, idx in enumerate(sorted_indices):
        size_sorted_list[idx] = i
    size_sorted_list =  [v/np.max(size_sorted_list) for v in size_sorted_list]
    
    alpha_grad = [grad[i] if v>=alpha else 0 for i,v in  enumerate(size_sorted_list)]
    # print(alpha_grad)
    # alpha_grad = [v if v>=alpha else 0 for v in grad]
    finalLs = []
    for i in indexLs:
        subGrad = alpha_grad[i[0]:i[1]]
        flag = subGrad.count(0)
        if flag <= int(len(subGrad)/2)-1:
            subSeq = list(seq[i[0]:i[1]])
            for j in range(len(subSeq)):
                if subGrad[j] == 0:
                    subSeq[j] = 'X'
            if subSeq[0]!= 'X' and subSeq[-1]!= 'X':
                finalLs.append(''.join(subSeq))


    return finalLs

def find_motif(input_string,grad,alpha):
    result = get_substring_indices(input_string)
    ls = filterSubstring(grad,alpha=alpha, indexLs = result, seq=input_string)

    return ls


# 使用示例
input_string = "abcdesdkjlgf"
grad = [.4,.6,.4,0,.4,.6,.4,0,0,.4,.6,.4,]
result = get_substring_indices(input_string)
ls = filterSubstring(grad,alpha = 0.4, indexLs = result, seq=input_string)
print(ls)
ls1 = find_motif(input_string=input_string,grad = grad,alpha=0.4)
print(ls1)

['bXXesd', 'esdXXl', 'esdXXlg', 'esdXXlgf', 'sdXXlg', 'sdXXlgf', 'dXXlgf']
['bXXesd', 'esdXXl', 'esdXXlg', 'esdXXlgf', 'sdXXlg', 'sdXXlgf', 'dXXlgf']


In [2]:
# 添加NBT attention模型
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '2,3'
import torch

import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker
import math

# 下面这几行是为了调用NBT att模型
from keras.models import load_model
from numpy import loadtxt, savetxt
import re

from Attention import Attention_layer


import warnings
warnings.filterwarnings("ignore")

import requests
import re
import json
from bs4 import BeautifulSoup

url = "https://pepcalc.com/ppc.php"

mydata = json.loads(
    '{"hideInputFields": "no","nTerm": "(NH2-)","cTerm": "(-COOH)","aaCode": 0,"disulphideBonds": "","sequence": ""}')

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Content-Length': '<calculated when request is sent>'
}


mydict = {'A':0,'C':1,'D':2,'E':3,'F':4,'G':5,'H':6,'I':7,'K':8,'L':9,'M':10,'N':11,'P':12,'Q':13,'R':14,'S':15,'T':16,'V':17,'W':18,'Y':19}
myInvDict = dict([val, key] for key, val in mydict.items())
sigmoid = torch.sigmoid


NBTdict = {'A':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,'L':10,'M':11,'N':12,'P':13,'Q':14,'R':15,'S':16,'T':17,'V':18,'W':19,'Y':20}




MAX_MIC = math.log10(8192)
max_mic_buffer = 0.1
My_MAX_MIC = math.log10(600)


def CosineSimilarity(tensor_1, tensor_2):
    tensor_1 = tensor_1.squeeze()
    tensor_2 = tensor_2.squeeze()

    normalized_tensor_1 = tensor_1 / tensor_1.norm(dim=-1, keepdim=True)
    normalized_tensor_2 = tensor_2 / tensor_2.norm(dim=-1, keepdim=True)
    return (normalized_tensor_1 * normalized_tensor_2).sum()
    
def seq2num(seq):
    
    seqlist = list(seq)
    # print(seq)
    length = len(seq)
    result = re.findall(r'[BJOUXZ]',seq)
    # print(result)
    # 如果序列中有这几个氨基酸，则返回空
    if result:
        return 

    # 否则正常返回
    else:
        numlist = [NBTdict[char.upper()] for char in seqlist]
        
        zeroPad = [0 for i in range(300-length)]
        zeroPad.extend(numlist)
        zeroPad = np.array(zeroPad)
        
        return zeroPad


def dataProcessPipeline(seq):
    # 本函数先把序列转化为0-19组成的序列，然后onehot变化，再padding
    # 同时返回padding后的序列以及mask
    #print('ori seq',seq)
    testest = seq
    num_seq = [mydict[character.upper()] for character in seq]

    seq = np.array(num_seq,dtype=int)
    len = seq.shape[0]
    torch_seq = torch.tensor(seq)
    if torch.sum(torch_seq[torch_seq<0])!=0:
        print(torch_seq[torch_seq<0])
        print('wrong seq:',seq)
        print(testest)
    onehotSeq = torch.nn.functional.one_hot(torch_seq,num_classes=20)
    #onehotSeq = torch.nn.functional.one_hot(c
    pad = torch.nn.ZeroPad2d(padding=(0,0,0,100-len))
    mask = np.zeros(100,dtype = int)
    mask[len:]=1
    mask = torch.tensor(mask)

    pad_seq = pad(onehotSeq) 
    
    
    return pad_seq,mask


def num2onehot(array2d):
    result = torch.zeros_like(array2d)
    index = torch.argmax(array2d,dim = -1)
    for i in range(index.shape[0]):
        result[i,index[i]] = 1

    return result


class TrainDataset(Dataset):
    def __init__(self,data_path,transform = dataProcessPipeline):
        df = pd.read_csv(data_path,header=0)
        
        
        print(str(df.shape)+'\n')
        # df = df[df['Length']<=100]
        self.df = df
        # id = self.df['Length']<100
        # self.df = self.df[id]
        # print(self.df.shape)
        #self.df = self.df.ix[1:]
        # self.seqs = list(self.df['Sequence'])
        self.seqs = list(self.df['sequence'])

        #print(self.seqs.shape)
        self.values = self.df['value']
        # 数据集的单边阈值设置
        self.values[self.values>MAX_MIC] = MAX_MIC
        self.values = list(self.values)
        #print(self.labels.shape)
        self.transform = transform


    def __getitem__(self,idex):
        seq = self.seqs[idex]
        num_seq, mask = self.transform(seq)
        label = self.values[idex]


        return num_seq, mask, label, seq

    def __len__(self):
        return len(self.seqs)


class TestDataset(Dataset):
    def __init__(self,data_path,transform = dataProcessPipeline):
        self.df = pd.read_csv(data_path,header=0)
        self.seqs = self.df['Sequence']

        self.transform = transform


    def __getitem__(self,idex):
        seq = self.seqs[idex]
        num_seq, mask = self.transform(seq)

        return num_seq, mask, seq

    def __len__(self):
        return len(self.seqs)



class PositionalEncoding(nn.Module):
     def __init__(self, len, d_model=20, dropout=0):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(len, d_model)
        position = torch.arange(0, len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                                * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        #pe = pe.unsqueeze(0).transpose(0, 1)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

     def forward(self, x):
        x = x + self.pe
        #x = x + self.pe[:,:x.size(0), :]
        return x



pe = PositionalEncoding(len=100,d_model = 20)


class AttentionNetwork(nn.Module):
    
    def __init__(self,batch_size=128,embedding_size=20,num_tokens=100,num_classes=1,num_heads=4):
        super(AttentionNetwork,self).__init__()
        self.pe = PositionalEncoding(len=num_tokens,d_model = embedding_size)
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.num_tokens = num_tokens
        self.num_classes = num_classes
        self.num_heads = num_heads
        # self.hidden1 = 20
        self.hidden1 = 20
        self.hidden2 = 60
        self.hidden3 = 20
        self.dropout = 0.2

        # self.hidden2 = 100
        # self.hidden3 = 50
        # self.hidden4 = 20
        # self.dropout = 0.2
        self.relu = nn.ReLU()

        self.LN = nn.LayerNorm(normalized_shape = self.hidden1)
        self.fc1 = nn.Linear(self.embedding_size,self.hidden1)

        # self.qfc = nn.Linear(self.hidden1,self.hidden1)
        # self.kfc = nn.Linear(self.hidden1,self.hidden1)
        # self.vfc = nn.Linear(self.hidden1,self.hidden1)

        self.multihead_att = nn.MultiheadAttention(embed_dim=self.hidden1,num_heads = self.num_heads,batch_first=1,dropout=self.dropout)
        # 我这里先不用maxpool了
        self.flatten = nn.Flatten()
        self.fc2 = nn.Linear(self.hidden1*self.num_tokens,self.hidden2)
        self.fc3 = nn.Linear(self.hidden2,self.hidden3)
        self.new_fc4 = nn.Linear(self.hidden3,self.num_classes)
        # self.fc4 = nn.Linear(self.hidden3,self.hidden4)
        # self.fc5 = nn.Linear(self.hidden4,self.num_classes)
        self.dropout = nn.Dropout(self.dropout)
        self.softmax = nn.functional.softmax


    #这里的X当作对象 有 embedding 和 mask
    def forward(self,x,mask):
        x = self.pe(x)
        x = self.fc1(x)


        mask = mask.to(torch.bool)
        x, w1= self.multihead_att.forward(x,x,x,key_padding_mask=mask)
        # x, w1= self.multihead_att.forward(x,x,x,key_padding_mask=mask)
        # x, w1= self.multihead_att.forward(x,x,x,key_padding_mask=mask)


        # x = self.LN(x)
        # [N 100 20]

        
        
        #print(type(x),x.size)
        # x = torch.tensor(x)
        x = self.flatten(x)
        x = self.fc2(x)
        x = self.dropout(x)
        
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.new_fc4(x)


        #return x, w1, w2, w3, w4
        return x

Using TensorFlow backend.
/home/user2/anaconda3/envs/AMP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user2/anaconda3/envs/AMP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user2/anaconda3/envs/AMP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user2/anaconda3/envs/AMP/l

In [3]:
model_list = {
    # 
    'myAttention': '../../NewModel3_21_output1_Regression/myAttention/mean_0_changeTH_0_0/_AMP0.629_total0.543.pth',

}

test_model_list = model_list

opt_seqls = [

'LWWRKAKWKRKIAKRMIRVIGAAKI',                                                                                                                            
'KWLGAFGKMRKIAIRLRLKRKKAF',                                                                                                                             
'WWRLWKTLLKAPKKLTGLRRW',                                                                                                                             
'RKLKKLRWRAGMMYKYVKLK',
'MRFPWKHWWKKWKWWWKKKR',
'MKKARFWWWVAWKKLLRKKA'
]

labels = [
'myAttention9',
'myAttention7',
'CNN8',
'myAttention9',
'myAttention13',
'myAttention9'
]

label_dict = {opt_seqls[i]:labels[i] for i in range(len(opt_seqls))}

import json
import os
import numpy as np

# 指定之前保存的 JSON 文件名
file_name = "/home/user2/pj/AMP_2/attention_model/test_motif/score/seq_score_9_7_reg.json"


with open(file_name, "r") as file:
    loaded_data = json.load(file)

score_seq_dict = {}
for item in loaded_data:
    seq = item['seq']
    # print(seq)
    score = np.array(item['score'])
    score_seq_dict[seq] = score
    # print(score.shape)
    # draw_weight2(score,seq,'layer1')


df_ls = []
from matplotlib.colors import Normalize
norm = Normalize(vmin=-0.2, vmax=0.2)
for seq in opt_seqls:
    tseq = seq

    # ModelNameList = ['CNN','Transformer','myAttention','RCNN']
    ModelNameList = model_list.keys()

    gradient = {}
    

    oriseq = tseq

    # 在当前目录生成 该序列的csv 文件
    df = pd.DataFrame(columns = ['Sequence','Length','label'])
    items = [{'Sequence':oriseq,'Length':len(oriseq)}]
    df = df.append(items,ignore_index = 1)
    df.to_csv(oriseq+'.csv',index = False)


    SeqPath = oriseq+'.csv'



    testData1 = TrainDataset(data_path = r'../../myRegressionData/all_balance/mean/test.csv')
    test_loader1 = DataLoader(dataset=testData1, batch_size=4,drop_last=True)


    for modelName in ModelNameList:

        # modelName = 'myAttention'  # to change
        iternum = 0

        testData = TestDataset(data_path = SeqPath)
        test_loader = DataLoader(dataset=testData, batch_size=1)
        attmodel = torch.load(model_list[modelName])


        attmodel.cuda()
        attmodel.zero_grad()

        print(modelName,"_V2:")

            
        for data in test_loader: #序列优化 stratergy 1: 全局都用ensamble作为优化目标
            resultList = []
            # ensamble_values = []
            resultSeq = [oriseq]
            outMIC = []
            # attmodel.zero_grad()
            inputs,masks, seqs = data

            inputs = inputs.float()
            masks = masks.float()
            
            # inputs = inputs.cuda()
            # inputs.requires_grad = True
            masks = masks.cuda()
            print(seqs[0])

            if modelName == 'RCNN':
                attmodel.train()
            else:
                attmodel.eval()
    
            attmodel.zero_grad()
            stdev_spread = 0.1
            n_samples = 25
            x = inputs[0].detach()
            stdev = stdev_spread * (torch.max(x)-torch.min(x))
            x = x.numpy() 
            total_grad = np.zeros_like(x)
            for i in range(n_samples):

                    # final.append(xx)
                noise = np.random.normal(0,stdev,x.shape).astype(np.float32)
                x_plus_noise = x + noise
                
                x_plus_noise = torch.from_numpy(x_plus_noise).cuda()
                # x_plus_noise = x_plus_noise + 0.1
                x_plus_noise[masks[0]==1] = 0
                x_plus_noise = Variable(torch.unsqueeze(x_plus_noise,dim=0), requires_grad = True)
                # final.append(x_plus_noise)
                if i==0:
                    x_plus_noise = Variable(inputs.cuda(), requires_grad = True)
            
                if modelName == 'lstm_att' or modelName == 'RCNN'or modelName == 'CNN' or modelName == 'Transformer':
                    out = attmodel(x_plus_noise)
                else:
                    out = attmodel(x_plus_noise,masks)

                out = out.cpu()
                if 'RCNN' in modelName:
                    out = out.unsqueeze(0)

                conloss = -out

                conloss.backward()
                grad = x_plus_noise.grad
                # if i==0:
                #     print(grad)
                colindex = masks[0]==1
                grad[0][masks[0]==1] = 0
                grad = grad[0].cpu().numpy()
                
                total_grad += grad


            avg_grad = total_grad/n_samples
            myIndex = [mydict[v] for v in seqs[0]]

            mylen = 100-colindex.sum()
            # print(mylen)
            grad = avg_grad[:mylen]
            # grad = [grad[k,myIndex[k]] for k in range(len(seqs[0]))]
            mag = np.sum(abs(grad),axis=-1).tolist()

            # print(np.sum(grad))

            value = [grad[k,myIndex[k]] for k in range(len(seqs[0]))]
            value = [1 if v>=0 else -1 for v in value]
            # print(value)
            # max_grad = np.max(grad,axis=-1).tolist()
            # # grad  =  [value[i]/max_grad[i] for i in range(len(grad))]
            grad = [mag[k]*value[k] for k in range(len(value))]
            # grad = list(np.exp(grad))

            absGrad = np.sum(abs(np.array(grad)))
            grad = [v/absGrad for v in grad]
            # grad = [ v/sum(grad) for v in grad]
            gradient[modelName] = grad
    seq_dict = {'sequence': seq,'model':label_dict[seq]}
    for k in range(2,11):
        alpha = k/10
        ls = find_motif(tseq,grad,alpha)
        print(tseq,alpha,':',ls)
        print(sum(abs(np.array(grad))))
        seq_dict[alpha] = ls

    df_ls.append(seq_dict)


    # print(gradient)
    # grad = gradient['myAttention']
    # alpha = 0.05
    # ls = find_motif(tseq,grad,alpha)
    # print(tseq,':',ls)
    # print(sum(abs(np.array(grad))))


(1312, 3)

myAttention _V2:
LWWRKAKWKRKIAKRMIRVIGAAKI
LWWRKAKWKRKIAKRMIRVIGAAKI 0.2 : ['LWWXK', 'LWWXKA', 'LWWXKAK', 'LWWXKAKW', 'WWXK', 'WWXKA', 'WWXKAK', 'WWXKAKW', 'WWXKAKWK', 'WXKA', 'WXKAK', 'WXKAKW', 'WXKAKWK', 'WXKAKWKR', 'KAKW', 'KAKWK', 'KAKWKR', 'KAKWKRK', 'KAKWKRKI', 'AKWK', 'AKWKR', 'AKWKRK', 'AKWKRKI', 'KWKR', 'KWKRK', 'KWKRKI', 'KWKRKIXK', 'WKRK', 'WKRKI', 'WKRKIXK', 'WKRKIXKR', 'KRKI', 'KRKIXK', 'KRKIXKR', 'KRKIXKRM', 'RKIXK', 'RKIXKR', 'RKIXKRM', 'RKIXKRMI', 'KIXK', 'KIXKR', 'KIXKRM', 'KIXKRMI', 'KIXKRMIR', 'IXKR', 'IXKRM', 'IXKRMI', 'IXKRMIR', 'IXKRMIRV', 'KRMI', 'KRMIR', 'KRMIRV', 'KRMIRVI', 'KRMIRVIG', 'RMIR', 'RMIRV', 'RMIRVI', 'RMIRVIG', 'MIRV', 'MIRVI', 'MIRVIG', 'IRVI', 'IRVIG', 'RVIG', 'RVIGXXXI']
1.0000000000000002
LWWRKAKWKRKIAKRMIRVIGAAKI 0.3 : ['LWWXK', 'LWWXKA', 'LWWXKAK', 'LWWXKAKW', 'WWXK', 'WWXKA', 'WWXKAK', 'WWXKAKW', 'WWXKAKWK', 'WXKA', 'WXKAK', 'WXKAKW', 'WXKAKWK', 'KAKW', 'KAKWK', 'KAKWKXK', 'KAKWKXKI', 'AKWK', 'AKWKXK', 'AKWKXKI', 'KWKXK', 'KWKXKI',

In [4]:
df = pd.DataFrame(df_ls)
print(df)
df.to_csv('./6peptid_sign_only_threshold_rank_0.2.csv',index=0)

                    sequence          model  \
0  LWWRKAKWKRKIAKRMIRVIGAAKI   myAttention9   
1   KWLGAFGKMRKIAIRLRLKRKKAF   myAttention7   
2      WWRLWKTLLKAPKKLTGLRRW           CNN8   
3       RKLKKLRWRAGMMYKYVKLK   myAttention9   
4       MRFPWKHWWKKWKWWWKKKR  myAttention13   
5       MKKARFWWWVAWKKLLRKKA   myAttention9   

                                                 0.2  \
0  [LWWXK, LWWXKA, LWWXKAK, LWWXKAKW, WWXK, WWXKA...   
1  [KWLG, KWLGA, KWLGAF, KWLGAFG, KWLGAFGK, WLGA,...   
2  [WWRL, WWRLW, WWRLWK, WWRLWKXL, WRLW, WRLWK, W...   
3  [KXKK, KXKKXR, KXKKXRW, KXKKXRWR, KKXR, KKXRW,...   
4  [MRFXW, MRFXWK, MRFXWKH, MRFXWKHW, RFXW, RFXWK...   
5  [MKKA, MKKAR, MKKARF, MKKARFW, MKKARFWW, KKAR,...   

                                                 0.3  \
0  [LWWXK, LWWXKA, LWWXKAK, LWWXKAKW, WWXK, WWXKA...   
1  [WLGA, WLGAF, WLGAFG, LGAF, LGAFG, LGAFGXXR, G...   
2  [WWRXW, WWRXWK, WWRXWKXL, WRXW, WRXWK, WRXWKXL...   
3  [KKXXWR, KKXXWRXG, KXXWRXGM, WRXG, WRXGM, WRXG... 

In [6]:
import numpy as np

def get_substring_indices(input_string):
    # 输入一个字符串，输出一个列表，列表中的元素为该字符串的子串的index区间
    indices = []
    length = len(input_string)
    
    for i in range(length):
        for j in range(i+4,  min(i+9, length+1)):
            indices.append([i, j])
    
    return indices

def filterSubstring(grad:list, alpha:float, indexLs:list, seq:str):

    sorted_indices = sorted(range(len(grad)), key=lambda i: grad[i])
    size_sorted_list = [0] * len(grad)

    for i, idx in enumerate(sorted_indices):
        size_sorted_list[idx] = i
    size_sorted_list =  [v/np.max(size_sorted_list) for v in size_sorted_list]
    
    alpha_grad = [grad[i] if v>=alpha else 0 for i,v in  enumerate(size_sorted_list)]
    # print(alpha_grad)
    # alpha_grad = [v if v>=alpha else 0 for v in grad]
    finalLs = []
    for i in indexLs:
        subGrad = alpha_grad[i[0]:i[1]]
        flag = subGrad.count(0)
        if flag <= int(len(subGrad)/2)-1:
            subSeq = list(seq[i[0]:i[1]])
            for j in range(len(subSeq)):
                if subGrad[j] == 0:
                    subSeq[j] = 'X'
            if subSeq[0]!= 'X' and subSeq[-1]!= 'X':
                finalLs.append(''.join(subSeq))


    return finalLs

def find_motif(input_string,grad,alpha):
    result = get_substring_indices(input_string)
    ls = filterSubstring(grad,alpha=alpha, indexLs = result, seq=input_string)

    return ls


# 使用示例
input_string = "abcdesdkjlgf"
grad = [.4,.6,.4,0,.4,.6,.4,0,0,.4,.6,.4,]
result = get_substring_indices(input_string)
ls = filterSubstring(grad,alpha = 0.4, indexLs = result, seq=input_string)
print(ls)
ls1 = find_motif(input_string=input_string,grad = grad,alpha=0.4)
print(ls1)

['bXXesd', 'esdXXl', 'esdXXlg', 'esdXXlgf', 'sdXXlg', 'sdXXlgf', 'dXXlgf']
['bXXesd', 'esdXXl', 'esdXXlg', 'esdXXlgf', 'sdXXlg', 'sdXXlgf', 'dXXlgf']


In [7]:
import json
file_name = '/home/user2/pj/AMP_2/attention_model/test_motif/score/seq_score_9_7_reg.json'
with open(file_name, "r") as file:
    loaded_data = json.load(file)


opt_seqls = [

'LWWRKAKWKRKIAKRMIRVIGAAKI',                                                                                                                            
'KWLGAFGKMRKIAIRLRLKRKKAF',                                                                                                                             
'WWRLWKTLLKAPKKLTGLRRW',                                                                                                                             
'RKLKKLRWRAGMMYKYVKLK',
'MRFPWKHWWKKWKWWWKKKR',
'MKKARFWWWVAWKKLLRKKA'
]

labels = [
'myAttention9',
'myAttention7',
'CNN8',
'myAttention9',
'myAttention13',
'myAttention9'
]

label_dict = {opt_seqls[i]:labels[i] for i in range(len(opt_seqls))}


scoreDict = {}
for item in loaded_data:
    seq = item['seq']
    score = item['score']
    scoreDict[seq] = score

df_ls = []
for seq in opt_seqls:
    score = scoreDict[seq]
    seq_dict = {'sequence': seq,'model':label_dict[seq]}
    for k in range(2,11):
        alpha = k/10
        ls = find_motif(seq,score,alpha)
        print(seq,alpha,':',ls)
        print(sum(abs(np.array(grad))))
        seq_dict[alpha] = ls
    df_ls.append(seq_dict)



LWWRKAKWKRKIAKRMIRVIGAAKI 0.2 : ['LXXRKA', 'LXXRKAK', 'LXXRKAKW', 'RKAK', 'RKAKW', 'RKAKWK', 'RKAKWKXK', 'KAKW', 'KAKWK', 'KAKWKXK', 'KAKWKXKI', 'AKWK', 'AKWKXK', 'AKWKXKI', 'AKWKXKIA', 'KWKXK', 'KWKXKI', 'KWKXKIA', 'KWKXKIAK', 'WKXK', 'WKXKI', 'WKXKIA', 'WKXKIAK', 'KXKI', 'KXKIA', 'KXKIAK', 'KXKIAKXM', 'KIAK', 'KIAKXM', 'KIAKXMI', 'IAKXM', 'IAKXMI', 'IAKXMIXV', 'AKXM', 'AKXMI', 'AKXMIXV', 'KXMI', 'KXMIXV', 'KXMIXVXG', 'MIXV', 'MIXVXG']
4.2
LWWRKAKWKRKIAKRMIRVIGAAKI 0.3 : ['KAKXK', 'KAKXKXK', 'KAKXKXKI', 'AKXK', 'AKXKXK', 'AKXKXKI', 'AKXKXKIA', 'KXKXKI', 'KXKXKIA', 'KXKXKIAK', 'KXKI', 'KXKIA', 'KXKIAK', 'KXKIAKXM', 'KIAK', 'KIAKXM', 'KIAKXMI', 'IAKXM', 'IAKXMI', 'AKXM', 'AKXMI', 'KXMI']
4.2
LWWRKAKWKRKIAKRMIRVIGAAKI 0.4 : ['KAKXK', 'KAKXKXK', 'KAKXKXKI', 'AKXK', 'AKXKXK', 'AKXKXKI', 'AKXKXKIA', 'KXKXKI', 'KXKXKIA', 'KXKXKIAK', 'KXKI', 'KXKIA', 'KXKIAK', 'KXKIAKXM', 'KIAK', 'KIAKXM', 'KIAKXMI', 'IAKXM', 'IAKXMI', 'AKXM', 'AKXMI', 'KXMI']
4.2
LWWRKAKWKRKIAKRMIRVIGAAKI 0.5 : ['KAKXK', 'KA

In [9]:
import pandas as pd
df = pd.DataFrame(df_ls)
print(df)
df.to_csv('./6peptid_sign_only_threshold_rank_0.2_attention_score.csv',index=0)

                    sequence          model  \
0  LWWRKAKWKRKIAKRMIRVIGAAKI   myAttention9   
1   KWLGAFGKMRKIAIRLRLKRKKAF   myAttention7   
2      WWRLWKTLLKAPKKLTGLRRW           CNN8   
3       RKLKKLRWRAGMMYKYVKLK   myAttention9   
4       MRFPWKHWWKKWKWWWKKKR  myAttention13   
5       MKKARFWWWVAWKKLLRKKA   myAttention9   

                                                 0.2  \
0  [LXXRKA, LXXRKAK, LXXRKAKW, RKAK, RKAKW, RKAKW...   
1  [KWLG, KWLGA, KWLGAF, KWLGAFG, KWLGAFGK, WLGA,...   
2  [WWRL, WWRLW, WWRLWK, WWRLWKT, WWRLWKTL, WRLW,...   
3  [RKLK, RKLKK, RKLKKL, RKLKKLR, KLKK, KLKKL, KL...   
4  [MRFP, MRFPW, MRFPWK, MRFPWKH, MRFPWKHW, RFPW,...   
5  [MKKXR, MKKXRF, MKKXRFW, MKKXRFWW, KKXR, KKXRF...   

                                                 0.3  \
0  [KAKXK, KAKXKXK, KAKXKXKI, AKXK, AKXKXK, AKXKX...   
1  [KWLG, KWLGA, KWLGAF, KWLGAFG, KWLGAFGK, WLGA,...   
2  [WWXL, WWXLW, WWXLWK, WWXLWKT, WWXLWKTL, WXLW,...   
3  [RKXK, RKXKK, RKXKKL, RKXKKLR, KXKK, KXKKL, KX... 

In [8]:
model_list = {
    # 
    'myAttention': '../../NewModel3_21_output1_Regression/myAttention/mean_0_changeTH_0_0/_AMP0.629_total0.543.pth',

}

test_model_list = model_list

opt_seqls = [

'MKKARFWWWVAWKKLLRKKA'
]

labels = [
'myAttention9',
'myAttention7',
'CNN8',
'myAttention9',
'myAttention13',
'myAttention9'
]

label_dict = {opt_seqls[i]:labels[i] for i in range(len(opt_seqls))}

import json
import os
import numpy as np

# 指定之前保存的 JSON 文件名
file_name = "/home/user2/pj/AMP_2/attention_model/test_motif/score/seq_score_9_7_reg.json"


with open(file_name, "r") as file:
    loaded_data = json.load(file)

score_seq_dict = {}
for item in loaded_data:
    seq = item['seq']
    # print(seq)
    score = np.array(item['score'])
    score_seq_dict[seq] = score
    # print(score.shape)
    # draw_weight2(score,seq,'layer1')


df_ls = []
from matplotlib.colors import Normalize
norm = Normalize(vmin=-0.2, vmax=0.2)
for seq in opt_seqls:
    tseq = seq

    # ModelNameList = ['CNN','Transformer','myAttention','RCNN']
    ModelNameList = model_list.keys()

    gradient = {}
    

    oriseq = tseq

    # 在当前目录生成 该序列的csv 文件
    df = pd.DataFrame(columns = ['Sequence','Length','label'])
    items = [{'Sequence':oriseq,'Length':len(oriseq)}]
    df = df.append(items,ignore_index = 1)
    df.to_csv(oriseq+'.csv',index = False)


    SeqPath = oriseq+'.csv'



    testData1 = TrainDataset(data_path = r'../../myRegressionData/all_balance/mean/test.csv')
    test_loader1 = DataLoader(dataset=testData1, batch_size=4,drop_last=True)


    for modelName in ModelNameList:

        # modelName = 'myAttention'  # to change
        iternum = 0

        testData = TestDataset(data_path = SeqPath)
        test_loader = DataLoader(dataset=testData, batch_size=1)
        attmodel = torch.load(model_list[modelName])


        attmodel.cuda()
        attmodel.zero_grad()

        print(modelName,"_V2:")

            
        for data in test_loader: #序列优化 stratergy 1: 全局都用ensamble作为优化目标
            resultList = []
            # ensamble_values = []
            resultSeq = [oriseq]
            outMIC = []
            # attmodel.zero_grad()
            inputs,masks, seqs = data

            inputs = inputs.float()
            masks = masks.float()
            
            # inputs = inputs.cuda()
            # inputs.requires_grad = True
            masks = masks.cuda()
            print(seqs[0])

            if modelName == 'RCNN':
                attmodel.train()
            else:
                attmodel.eval()
    
            attmodel.zero_grad()
            stdev_spread = 0.1
            n_samples = 25
            x = inputs[0].detach()
            stdev = stdev_spread * (torch.max(x)-torch.min(x))
            x = x.numpy() 
            total_grad = np.zeros_like(x)
            for i in range(n_samples):

                    # final.append(xx)
                noise = np.random.normal(0,stdev,x.shape).astype(np.float32)
                x_plus_noise = x + noise
                
                x_plus_noise = torch.from_numpy(x_plus_noise).cuda()
                # x_plus_noise = x_plus_noise + 0.1
                x_plus_noise[masks[0]==1] = 0
                x_plus_noise = Variable(torch.unsqueeze(x_plus_noise,dim=0), requires_grad = True)
                # final.append(x_plus_noise)
                if i==0:
                    x_plus_noise = Variable(inputs.cuda(), requires_grad = True)
            
                if modelName == 'lstm_att' or modelName == 'RCNN'or modelName == 'CNN' or modelName == 'Transformer':
                    out = attmodel(x_plus_noise)
                else:
                    out = attmodel(x_plus_noise,masks)

                out = out.cpu()
                if 'RCNN' in modelName:
                    out = out.unsqueeze(0)

                conloss = -out

                conloss.backward()
                grad = x_plus_noise.grad
                # if i==0:
                #     print(grad)
                colindex = masks[0]==1
                grad[0][masks[0]==1] = 0
                grad = grad[0].cpu().numpy()
                
                total_grad += grad


            avg_grad = total_grad/n_samples
            myIndex = [mydict[v] for v in seqs[0]]

            mylen = 100-colindex.sum()
            # print(mylen)
            grad = avg_grad[:mylen]
            # grad = [grad[k,myIndex[k]] for k in range(len(seqs[0]))]
            mag = np.sum(abs(grad),axis=-1).tolist()

            # print(np.sum(grad))

            value = [grad[k,myIndex[k]] for k in range(len(seqs[0]))]
            # print(value)
            # max_grad = np.max(grad,axis=-1).tolist()
            # # grad  =  [value[i]/max_grad[i] for i in range(len(grad))]
            grad = [mag[k]*value[k] for k in range(len(value))]
            # grad = list(np.exp(grad))

            absGrad = np.sum(abs(np.array(grad)))
            grad = [v/absGrad for v in grad]
            # grad = [ v/sum(grad) for v in grad]
            gradient[modelName] = grad
    seq_dict = {'sequence': seq,'model':label_dict[seq]}
    print(gradient)
    for k, v in gradient.items():
        for id in range(len(seq)):
            print(str(seq[id])+":",v[id])
    # for k in range(11):
    #     alpha = k/100
    #     ls = find_motif(tseq,grad,alpha)
    #     print(tseq,':',ls)
    #     print(sum(abs(np.array(grad))))
    #     seq_dict[alpha] = ls

    # df_ls.append(seq_dict)


    # print(gradient)
    # grad = gradient['myAttention']
    # alpha = 0.05
    # ls = find_motif(tseq,grad,alpha)
    # print(tseq,':',ls)
    # print(sum(abs(np.array(grad))))


(1312, 3)

myAttention _V2:
MKKARFWWWVAWKKLLRKKA
{'myAttention': [0.153208447175486, 0.020525608204773126, 0.0013144287048440777, 0.027304417124256988, 0.36332461671415434, 0.048659490991791, 0.09964009626999414, 0.03316174482352915, -0.007934305219482416, -0.02032431014211555, 0.0031446601535416435, 0.017557999498349297, -0.017903503413233348, 0.003018150422452292, 0.0658418824121672, 0.05011999926403741, -0.000879505613447257, 0.016463834797143473, 0.029862093833343734, -0.019810905221857633]}
M: 0.153208447175486
K: 0.020525608204773126
K: 0.0013144287048440777
A: 0.027304417124256988
R: 0.36332461671415434
F: 0.048659490991791
W: 0.09964009626999414
W: 0.03316174482352915
W: -0.007934305219482416
V: -0.02032431014211555
A: 0.0031446601535416435
W: 0.017557999498349297
K: -0.017903503413233348
K: 0.003018150422452292
L: 0.0658418824121672
L: 0.05011999926403741
R: -0.000879505613447257
K: 0.016463834797143473
K: 0.029862093833343734
A: -0.019810905221857633
